# 5. 파일 저장

## 이미지 저장

In [4]:
import cv2
img = cv2.imread("img.jpg", cv2.IMREAD_GRAYSCALE) # 흑백으로 이미지 불러오기
cv2.imwrite("img_save.png", img)

True

## 동영상 저장

In [6]:
import cv2
cap = cv2.VideoCapture('video.mp4')

# 코덱 정의
fourcc = cv2.VideoWriter_fourcc(*"DIVX")

width = round(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = round(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS) # * 2, 2배 속도로 저장

out = cv2.VideoWriter('output.avi', fourcc, fps, (width, height))
# 저장 파일명, 코덱, fps, 크기, (width, height)

while cap.isOpened(): # 동영상 파일이 올바로 열렸는지?
    ret, frame = cap.read() # ret : 성공 여부, frame : 받아온 이미지 (프레임)
    if not ret:
        break

    out.write(frame)
    cv2.imshow("video", frame) # 영상 데이터만 저장 소리 X
    if cv2.waitKey(1) == ord("q"): # 영상 속도
        break

out.release() # 자원 해제
cap.release()
cv2.destroyAllWindows()
for i in range (1,5): # m1 창 안 닫힐때
    cv2.waitKey(1)